In [1]:
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.ops import clip_ops
import tensorflow as tf

In [2]:
_EPSILON = 1e-7

def categorical_focal_loss(target, output, alpha, gamma=2, axis=-1):
    """Categorical focal loss between an output tensor and a target tensor.
    Arguments:
      target: A tensor of the same shape as `output`.
      
      output: A tensor resulting from a softmax
          (unless `from_logits` is True, in which
          case `output` is expected to be the logits).
      
      alpha: values for class imbalance
      
      gamma: focusing parameter used to calculate the modulating factor
      
      axis: Int specifying the channels axis. `axis=-1` corresponds to data
          format `channels_last', and `axis=1` corresponds to data format
          `channels_first`.
    Returns:
      Output tensor.
    Raises:
      ValueError: if `axis` is neither -1 nor one of the axes of `output`.
    """
    rank = len(output.shape)
    axis = axis % rank
    # scale preds so that the class probas of each sample sum to 1
    output = output / math_ops.reduce_sum(output, axis, True)
    # manual computation of crossentropy
    epsilon_ = ops.convert_to_tensor(_EPSILON, output.dtype.base_dtype)
    output = clip_ops.clip_by_value(output, epsilon_, 1. - epsilon_)
    
    return -math_ops.reduce_sum(tf.matmul( target * math_ops.log(output)*math_ops.pow(1-output, gamma), tf.transpose(alpha)), axis)

In [5]:
#Checking the loss function
target = tf.constant([[1., 0., 0.], [0., 0., 1.], [1., 0., 0.], [0., 1., 0.]])
output = tf.constant([[1., 0., 0.], [0.1, 0.2, 0.8], [0.7, 0.1, 0.2], [0.1, 0.9, 0.3]])
results = categorical_focal_loss(target, output, tf.constant([[0.2, 0.4, 0.4]]))
sess = tf.Session()
result = sess.run(results)
print(result)
sess.close()

[3.3881318e-22 9.4746538e-03 6.4201499e-03 1.3925669e-02]
